Шепелев Д.

# **Применение RNN для прогнозирования отношения в рецензиях на фильмы**

## **Импорты**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [36]:
import numpy as np
import pandas as pd

from collections import Counter

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

## **Подгрузка датасета**

In [6]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')

In [9]:
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


## **Подготовка данных**

### Формирование датасета

In [10]:
target = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices((df.values, target.values))

for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


2023-05-31 21:36:55.408769: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-05-31 21:36:55.408804: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: Proton
2023-05-31 21:36:55.408813: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: Proton
2023-05-31 21:36:55.408891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.105.17
2023-05-31 21:36:55.408913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.105.17
2023-05-31 21:36:55.408918: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 525.105.17
2023-05-31 21:36:55.410408: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network L

In [17]:
tf.random.set_seed(42)
ds_raw = ds_raw.shuffle(50000, reshuffle_each_iteration=False)

ds_raw_test = ds_raw.take(25000)
ds_raw_train_val = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_val.take(20000)
ds_raw_val = ds_raw_train_val.skip(20000)

### Поиск уникальных лексем

In [18]:
tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()

In [19]:
for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy()[0])
    token_counts.update(tokens)

In [20]:
print('Размер словаря: ', len(token_counts))

Размер словаря:  87136


### Кодирование уникальных лексем в виде целых чисел

In [21]:
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)

In [22]:
def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    return encoded_text, label

In [24]:
def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label],
                        Tout = (tf.int64, tf.int64))

In [27]:
ds_train = ds_raw_train.map(encode_map_fn)
ds_val = ds_raw_val.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

In [28]:
tf.random.set_seed(42)

for example in ds_train.shuffle(1000).take(5):
    print(f'Длина последовательности: {example[0].shape}')

Длина последовательности: (179,)
Длина последовательности: (298,)
Длина последовательности: (95,)
Длина последовательности: (123,)
Длина последовательности: (45,)


## **Разделение на пакеты**

In [29]:
ds_subset = ds_train.take(8)
for example in ds_subset:
    print('Размер индивидуального элемента: ', example[0].shape)

Размер индивидуального элемента:  (99,)
Размер индивидуального элемента:  (149,)
Размер индивидуального элемента:  (278,)
Размер индивидуального элемента:  (139,)
Размер индивидуального элемента:  (174,)
Размер индивидуального элемента:  (106,)
Размер индивидуального элемента:  (77,)
Размер индивидуального элемента:  (523,)


In [30]:
ds_batched = ds_subset.padded_batch(
    4, padded_shapes=([-1], [])
)
for batch in ds_batched:
    print('Размерность пакета: ', batch[0].shape)

Размерность пакета:  (4, 278)
Размерность пакета:  (4, 523)


In [32]:
train_data = ds_train.padded_batch(32, padded_shapes=([-1], []))
valid_data = ds_val.padded_batch(32, padded_shapes=([-1], []))
test_data = ds_test.padded_batch(32, padded_shapes=([-1], []))

In [33]:
for batch in train_data.take(5):
    print('Размерность пакета: ', batch[0].shape)

Размерность пакета:  (32, 821)
Размерность пакета:  (32, 729)
Размерность пакета:  (32, 1043)
Размерность пакета:  (32, 704)
Размерность пакета:  (32, 509)


## **Разработка модели**

In [37]:
embedding_dim=20
vocab_size = len(token_counts) + 2

tf.random.set_seed(42)

bi_lstm_model = tf.keras.Sequential([
    layers.Embedding(input_dim=vocab_size,
                     output_dim=embedding_dim,
                     name='embed_layer'), 
    layers.Bidirectional(
        layers.LSTM(64, name='lstm_layer'),
        name='bidir-lstm'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed_layer (Embedding)     (None, None, 20)          1742760   
                                                                 
 bidir-lstm (Bidirectional)  (None, 128)               43520     
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,794,601
Trainable params: 1,794,601
Non-trainable params: 0
_________________________________________________________________


**Компиляция модели (определения оптимизатора и функции потерь)**

In [38]:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy']
)

## **Обучение**

In [39]:
history = bi_lstm_model.fit(
    train_data,
    validation_data=valid_data,
    epochs=5)

Epoch 1/5
625/625 [==============================] - 143s 226ms/step - loss: 0.6518 - accuracy: 0.6069 - val_loss: 0.6910 - val_accuracy: 0.4956
Epoch 2/5
625/625 [==============================] - 141s 226ms/step - loss: 0.5827 - accuracy: 0.6863 - val_loss: 0.5199 - val_accuracy: 0.7206
Epoch 3/5
625/625 [==============================] - 146s 233ms/step - loss: 0.3783 - accuracy: 0.8462 - val_loss: 0.4542 - val_accuracy: 0.8066
Epoch 4/5
625/625 [==============================] - 144s 231ms/step - loss: 0.2280 - accuracy: 0.9171 - val_loss: 0.4201 - val_accuracy: 0.8338
Epoch 5/5
625/625 [==============================] - 143s 229ms/step - loss: 0.1265 - accuracy: 0.9590 - val_loss: 0.4659 - val_accuracy: 0.8324


## **Оценка на тестовой выборке**

In [40]:
test_results = bi_lstm_model.evaluate(test_data)
print('{:.2f}%'.format(test_results[1] * 100))

782/782 [==============================] - 48s 61ms/step - loss: 0.4676 - accuracy: 0.8331
83.31%
